In [3]:
#import required libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pl




In [4]:
#read the dataset

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [5]:
#Information about the dataset

print("Size of dataset is {train.shape}

(42000, 785)